In [25]:
from instances import get_instances
import numpy as np
import pandas as pd
from meta_genetique import genetic_algo
from greedy import greedy_knapsack
from heuristique_de_reparation import reparation_surrogate
from recherche_locale import recherche_locale_combinee
import time
import json
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

## Boucles sur tout les fichiers avec tout les algos

In [7]:
files = [f'instances/mknapcb{i}' for i in range(1, 10)]
df = pd.DataFrame(columns=['file','instance', 'opt_value', 'gap_genetic', 'gap_greedy', 'gap_surrogate', 'gap_rech_local_greedy', 'gap_rech_local_surrogate'])
with tqdm(total=len(files), desc="Traitement des fichiers", unit="fichier") as overall_progress:
    for file in files : 
        inst_file = file + '.txt'
        with open(file + '_sol.json', 'r') as f:
            opt_val_file = list(json.load(f).values())
        
        instances = get_instances(inst_file)
        with tqdm(total=len(instances), desc=f"Traitement de {file}", leave=True, unit="instance") as file_progress:
            for i in range(len(instances)):
                ins = instances[i]
                ins["opt_value"] = opt_val_file[i]
                
                c = ins["gains"]
                a = np.array(ins["ressources"])
                b = ins["quantite_ressources"]
                N = len(c)
                M = len(b)
                x_greedy, value_greedy = greedy_knapsack(c, a, b)
                x_surrogate, value_surroate = reparation_surrogate(N, M, a,b,c)
                x_rech_local_greed, value_rech_local_greed = recherche_locale_combinee(N, M, c, a, b, x_greedy)
                x_rech_local_surr, value_rech_local_surr = recherche_locale_combinee(N, M, c, a, b, x_surrogate)
                x_best, best_value = genetic_algo(N, M, c, a, b, max_iter=200, pop_size=100)
                infos = infos = {
                    'file': [file.split('/')[-1]],
                    'instance': [i],
                    'opt_value': [float(ins["opt_value"])],
                    'gap_genetic': [(best_value - float(ins["opt_value"])) / float(ins["opt_value"])],
                    'gap_greedy': [(value_greedy - float(ins["opt_value"])) / float(ins["opt_value"])],
                    'gap_surrogate': [(value_surroate - float(ins["opt_value"])) / float(ins["opt_value"])],
                    'gap_rech_local_greedy': [(value_rech_local_greed - float(ins["opt_value"])) / float(ins["opt_value"])],
                    'gap_rech_local_surrogate': [(value_rech_local_surr - float(ins["opt_value"])) / float(ins["opt_value"])]
                }
                df = pd.concat([df, pd.DataFrame(infos)])
                file_progress.update(1)
        overall_progress.update(1)

Traitement des fichiers: 100%|██████████| 9/9 [1:28:50<00:00, 592.23s/fichier] 


In [12]:
# absolute value of the gap
df = df.assign(gap_genetic = lambda x: abs(x['gap_genetic'])*100,
            gap_greedy = lambda x: abs(x['gap_greedy'])*100,
            gap_surrogate = lambda x: abs(x['gap_surrogate'])*100,
            gap_rech_local_greedy = lambda x: abs(x['gap_rech_local_greedy'])*100,
            gap_rech_local_surrogate = lambda x: abs(x['gap_rech_local_surrogate'])*100
             )

### générer code latex du tableau

In [23]:
file = "mknapcb9"
caption = "{"+ f"Résultats pour les instances du fichier {file}" + "}"
print( "\\begin{table}[]\n\centering")
print(df.query("file == @file").to_latex(index=False, formatters={"name": str.upper},
                  float_format="{:.1f}".format,))
print(f"\caption{caption}")
print("\label{tab:my_label}")
print("\end{table}")

\begin{table}[]
\centering
\begin{tabular}{llrrrrrr}
\toprule
file & instance & opt_value & gap_genetic & gap_greedy & gap_surrogate & gap_rech_local_greedy & gap_rech_local_surrogate \\
\midrule
mknapcb9 & 0 & 115868.0 & 6.9 & 5.3 & 6.0 & 5.3 & 1.9 \\
mknapcb9 & 1 & 114667.0 & 6.9 & 4.0 & 4.7 & 4.0 & 1.5 \\
mknapcb9 & 2 & 116661.0 & 4.9 & 3.5 & 5.5 & 3.5 & 1.8 \\
mknapcb9 & 3 & 115237.0 & 6.8 & 6.7 & 9.8 & 6.7 & 2.2 \\
mknapcb9 & 4 & 116353.0 & 8.0 & 6.9 & 8.2 & 6.9 & 1.6 \\
mknapcb9 & 5 & 115604.0 & 6.2 & 4.2 & 6.3 & 4.2 & 1.9 \\
mknapcb9 & 6 & 113952.0 & 7.3 & 6.6 & 8.2 & 6.6 & 1.6 \\
mknapcb9 & 7 & 114199.0 & 7.3 & 3.6 & 6.6 & 3.6 & 1.4 \\
mknapcb9 & 8 & 115247.0 & 5.3 & 4.0 & 5.4 & 4.0 & 1.7 \\
mknapcb9 & 9 & 116947.0 & 5.6 & 5.6 & 7.0 & 5.6 & 1.6 \\
mknapcb9 & 10 & 217995.0 & 11.1 & 2.5 & 3.9 & 2.5 & 0.8 \\
mknapcb9 & 11 & 214534.0 & 12.1 & 2.8 & 4.1 & 2.8 & 1.0 \\
mknapcb9 & 12 & 215854.0 & 11.0 & 2.2 & 3.0 & 2.2 & 1.0 \\
mknapcb9 & 13 & 217836.0 & 11.7 & 3.8 & 4.0 & 3.8 & 1.5 \

## Boucle gloutonne

In [ ]:
files = [f'instances/mknapcb{i}' for i in range(1, 10)]
df = pd.DataFrame(columns=['file','instance', 'opt_value', 'value_gloutonne', 'gap_gloutonne'])
with tqdm(total=len(files), desc="Traitement des fichiers", unit="fichier") as overall_progress:
    for file in files : 
        inst_file = file + '.txt'
        with open(file + '_sol.json', 'r') as f:
            opt_val_file = list(json.load(f).values())
        
        instances = get_instances(inst_file)
        with tqdm(total=len(instances), desc=f"Traitement de {file}", leave=True, unit="instance") as file_progress:
            for i in range(len(instances)):
                ins = instances[i]
                ins["opt_value"] = opt_val_file[i]
                
                c = ins["gains"]
                a = np.array(ins["ressources"])
                b = ins["quantite_ressources"]
                x_best, best_value = greedy_knapsack(c, a, b)
                infos = infos = {
                    'file': [file.split('/')[-1]],
                    'instance': [i],
                    'opt_value': [float(ins["opt_value"])],
                    'value_gloutonne': [best_value],
                    'gap_gloutonne':[ (float(ins["opt_value"]) - best_value) / float(ins["opt_value"])*100]
                }
                df = pd.concat([df, pd.DataFrame(infos)])
                file_progress.update(1)
        overall_progress.update(1)

\section{Métaheuristique: Algorithme génétique}
Nous avons choisi l'algorithme génétique comme métaheuristique pour résoudre ce problème. L'algorithme génétique se base sur le generation de solution en appliquant des opérations génétiques (croisement, mutation, selection) sur des individus (solutions) pour obtenir des solutions de plus en plus optimales. L'algorithme génétique que nous avons implementé genere d'abords plusieurs soluttions aléatoires, puis il les évalue en utilisant la fonction objectif, ensuite il selectionne les meilleurs solutions pour les croiser et les muter pour obtenir des solutions plus optimales. L'algorithme s'arrete lorsqu'il atteint un nombre maximal d'itérations ou lorsqu'il n'arrive pas à améliorer la solution pendant un certain nombre d'itérations.

L'algorithme génetique part d'une population de solutionspuis à travers les mechanismes suivant (mutation, croisement, selection) il génère une nouvelle population de solutions et ainsi de suite pour un nombre d'itérations donné, ou jusqu'a atteindre un critère d'arret.

\subsection{Génération de la population initiale}
Pour générer la population initiale, nous avons utilisé la méthode suivante:
- Générer une solution aléatoire
- Réparer la solution pour obtenir une solution feasible
- Générer les solutions voisines faisables en utilisant la distance de Hamming


\subsection{Mutation}
Pour la mutation, nous avons utilisé la méthode suivante:
- Inverser les bits de la solution
- Réparer la solution pour obtenir une solution feasible

\subsection{Croisement}
Pour le croisement, nous avons utilisé la méthode suivante:
- Choisir deux solutions avec la methode de roulette de fortune (les solutions avec les meilleurs valeurs de la fonction objectif ont plus de chance d'etre choisi)
- Echanger les bits des deux solutions pour avoir deux nouvelles solutions






In [27]:
files = [f'instances/mknapcb{i}' for i in range(1, 2)]
df = pd.DataFrame(columns=['file','instance', 'opt_value', 'gap_genetic', 'gap_rech_local_surrogate', 'time_genetic', 'time_rech_local_surr'])
with tqdm(total=len(files), desc="Traitement des fichiers", unit="fichier") as overall_progress:
    for file in files : 
        inst_file = file + '.txt'
        with open(file + '_sol.json', 'r') as f:
            opt_val_file = list(json.load(f).values())
        
        instances = get_instances(inst_file)
        with tqdm(total=len(instances), desc=f"Traitement de {file}", leave=True, unit="instance") as file_progress:
            for i in range(len(instances)):
                ins = instances[i]
                ins["opt_value"] = opt_val_file[i]
                
                c = ins["gains"]
                a = np.array(ins["ressources"])
                b = ins["quantite_ressources"]
                N = len(c)
                M = len(b)
                
                # x_greedy, value_greedy = greedy_knapsack(c, a, b)
                x_surrogate, value_surroate = reparation_surrogate(N, M, a,b,c)
                # x_rech_local_greed, value_rech_local_greed = recherche_locale_combinee(N, M, c, a, b, x_greedy)
                t = time.time()
                x_rech_local_surr, value_rech_local_surr = recherche_locale_combinee(N, M, c, a, b, x_surrogate)
                time_rech_local_surr = time.time() - t
                t = time.time()
                x_best, best_value = genetic_algo(N, M, c, a, b, max_iter=200, pop_size=100)
                time_genetic = time.time() - t
                infos = infos = {
                    'file': [file.split('/')[-1]],
                    'instance': [i],
                    'opt_value': [float(ins["opt_value"])],
                    'gap_genetic': [(best_value - float(ins["opt_value"])) / float(ins["opt_value"])],
                    'gap_rech_local_surrogate': [(value_rech_local_surr - float(ins["opt_value"])) / float(ins["opt_value"])],
                    'time_genetic': [time_genetic],
                    'time_rech_local_surr': [time_rech_local_surr]
                }
                df = pd.concat([df, pd.DataFrame(infos)])
                file_progress.update(1)
        overall_progress.update(1)

Traitement des fichiers: 100%|██████████| 1/1 [00:13<00:00, 13.20s/fichier]


In [29]:
files = ['instances/mknap1']
df = pd.DataFrame(columns=['file','instance', 'opt_value', 'gap_genetic', 'gap_rech_local_surrogate', 'time_genetic', 'time_rech_local_surr'])
with tqdm(total=len(files), desc="Traitement des fichiers", unit="fichier") as overall_progress:
    for file in files : 
        inst_file = file + '.txt'
        
        instances = get_instances(inst_file)
        with tqdm(total=len(instances), desc=f"Traitement de {file}", leave=True, unit="instance") as file_progress:
            for i in range(len(instances)):
                ins = instances[i]
                
                c = ins["gains"]
                a = np.array(ins["ressources"])
                b = ins["quantite_ressources"]
                N = len(c)
                M = len(b)
                # x_greedy, value_greedy = greedy_knapsack(c, a, b)
                x_surrogate, value_surroate = reparation_surrogate(N, M, a,b,c)
                # x_rech_local_greed, value_rech_local_greed = recherche_locale_combinee(N, M, c, a, b, x_greedy)
                t = time.time()
                x_rech_local_surr, value_rech_local_surr = recherche_locale_combinee(N, M, c, a, b, x_surrogate)
                time_rech_local_surr = time.time() - t
                t = time.time()
                x_best, best_value = genetic_algo(N, M, c, a, b, max_iter=200, pop_size=100)
                time_genetic = time.time() - t
                infos = infos = {
                    'file': [file.split('/')[-1]],
                    'instance': [i],
                    'opt_value': [float(ins["opt_value"])],
                    'gap_genetic': [(best_value - float(ins["opt_value"])) / float(ins["opt_value"])],
                    'gap_rech_local_surrogate': [(value_rech_local_surr - float(ins["opt_value"])) / float(ins["opt_value"])],
                    'time_genetic': [time_genetic],
                    'time_rech_local_surr': [time_rech_local_surr]
                }
                df = pd.concat([df, pd.DataFrame(infos)])
                file_progress.update(1)
        overall_progress.update(1)

Traitement des fichiers: 100%|██████████| 1/1 [00:00<00:00,  1.52fichier/s]


In [32]:
df.assign(
    gap_genetic = lambda x: abs(x['gap_genetic'])*100,
    gap_rech_local_surrogate = lambda x: abs(x['gap_rech_local_surrogate'])*100
            )

,file,instance,opt_value,gap_genetic,gap_rech_local_surrogate,time_genetic,time_rech_local_surr
0,mknap1,0,3800.0,0.000000,0.000000,0.057341,0.000000
0,mknap1,1,8706.1,4.240705,4.240705,0.037488,0.001010
0,mknap1,2,4015.0,0.249066,3.486924,0.063773,0.000000
0,mknap1,3,6120.0,1.797386,0.490196,0.083343,0.000000
0,mknap1,4,12400.0,0.000000,0.000000,0.099844,0.000000
0,mknap1,5,10618.0,4.012055,1.422113,0.099867,0.051781
0,mknap1,6,16537.0,0.804257,1.269880,0.123220,0.027036
